# Week 04 Assignment weather data

Welcome to week four of this course programming 1. You will to organise your data into the required format and apply smoothing. In this assignment we will work with weatherdata from the KNMI. A subset of weatherdata is for you available in the file: `KNMI_20181231.csv`. The data consist of several stations with daily weather data of several years. Your task is to make a plot similar to the plot below.


<img src="../images/weather.png" alt="drawing" width="400"/>


Furthermore the plot needs the following enhancements

1. proper titles and ticks
2. a slider widget selecting a particular year or all years
3. lines need to be smoothed
3. legends needs to be added

Use your creativity. Consider colors, alpha settings, sizes etc. 

Learning outcomes

- load, inspect and clean a dataset 
- reformat dataframes
- apply smoothing technologies
- visualize timeseries data

The assignment consists of 6 parts:

- [part 1: load the data](#0)
- [part 2: clean the data](#1)
- [part 3: reformat data](#2)
- [part 4: smooth the data](#3)
- [part 5: visualize the data](#4)
- [part 6: Challenge](#5)

Part 1 and 5 are mandatory, part 6 is optional (bonus)
To pass the assingnment you need to a score of 60%. 


---

<a name='0'></a>
## Part 1: Load the data

Either load the dataset `KNMI_20181231.csv` or `KNMI_20181231.txt.tsv`. The dataheaders contain spaces and are not very self explainable. Change this into more readable ones. Select data from station 270. Select only the mean, minimum and maximum temperature. The data should look something like this:


In [62]:
import pandas as pd
import numpy as np

df = pd.read_csv("/homes/tvanlieshout/jupyter_notebooks/Programming1/data/KNMI_20181231.txt.tsv",
                 sep=",", 
                 usecols=[0, 1, 2, 3, 4], 
                 names=["station", "Date", "Tmean", "Tmin", "Tmax"], 
                 comment="#")
station270 = df[df.station == 270]
station270

/homes/tvanlieshout/venvs/omicsproject/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (2,3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,station,Date,Tmean,Tmin,Tmax
97641,270,20000101,42,-4,79
97642,270,20000102,55,33,74
97643,270,20000103,74,49,89
97644,270,20000104,46,22,75
97645,270,20000105,41,14,56
...,...,...,...,...,...
104576,270,20181227,57,53,62
104577,270,20181228,71,58,81
104578,270,20181229,85,69,102
104579,270,20181230,80,68,90


---

<a name='1'></a>
## Part 2: Clean the data

The data ia not clean. There are empty cells in the dataframe which needs to be replaced with NaN's and the temperature is in centidegrees which needs to be transformed into degrees. The date field needs a datetime format. For visualization convience we would like to remove the leap year. Conduct the cleaning.

In [63]:
#replace cells with spaces to NaN
df = df.replace(r"^\s*$", np.nan, regex=True)
#change data formats
df.Date = pd.to_datetime(df['Date'].astype(str), format='%Y%m%d')
#change temperatures to celcius degrees
df.Tmean = pd.to_numeric(df.Tmean, errors='coerce') / 100
df.Tmin = pd.to_numeric(df.Tmin, errors='coerce') / 100
df.Tmax = pd.to_numeric(df.Tmax, errors='coerce') / 100
#remove leap year
df.sample(10)

,station,Date,Tmean,Tmin,Tmax
313288,377,2007-08-24,1.90,1.10,2.59
311525,377,2002-10-26,1.15,0.82,1.47
103582,270,2016-04-07,0.73,0.58,0.97
227246,319,2015-03-21,0.61,0.24,0.83
13905,215,2018-05-08,1.79,1.05,2.61
201345,313,2001-04-22,NaN,NaN,NaN
165827,286,2015-09-05,1.32,1.11,1.65
231003,323,2006-07-03,2.38,1.64,2.96
321434,380,2010-12-12,0.26,-0.25,0.59
278356,348,2007-01-04,0.93,0.71,0.99


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>pd.to_datetime(df['Date'].astype(str), format='%Y%m%d')</li>
    <li>regex for empty cells = `^\s*$` </li>
    <li>remove month == 2 & day == 29</li> 
</ul>
</details>

In [ ]:
#Test your outcome
df.info()

### Expected outcome

---

<a name='2'></a>
## Part 3: Reform your data

First we will split the data in data from 2018 and data before 2018. Best is to split this in two dataframes. 
Next we need for the non 2018 data the minimum values for each day and the maximum values for each day. So we look for the minimum value out of all january-01 minimum values (regardless the year). Create a dataframe with 365 days containing the ultimate minimum and the ultimate maximum per day. 


In [ ]:
def month_day(df_multipleyears):
    #your code to reform data here
    
    print(df_groupedbymonthday)

In [ ]:
#Test your code
def test_reformed(df)
    #
    df = df[(df.index.year > 2007) & (df.index.year < 2018)]
    month_day(df)

test_reformed(df)

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use the dt.month and dt.day to groupby</li>
</ul>
</details>

### Expected outcome
Note, the layout or names my differ, but the length should be 365 and the minimum values should be the same

---

<a name='3'></a>
## Part 4: Smooth the data

Make a function that takes an array or a dataframe column and returns an array of smoothed data. Explain in words why you choose a certain smoothing algoritm


In [ ]:
#your code here
#your motivation here

---

<a name='4'></a>
## Part 5: Visualize the data

Plot the mean temperature of the year 2018. Create a shaded band with the ultimate minimum values and the ultimate maximum values from the multi-year dataset. Add labels, titles and legends. Use proper ranges. Be creative to make the plot attractive. 



<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use from bokeh.models import Band</li>
    <li>use ColumnDataSource to parse data arrays</li>
    <li>look for xaxis tick formatters</li>
</ul>
</details>

---

<a name='5'></a>
## Part 6: Challenge

Make a widget in which you can select the year range for the multiyear set. Add this to your layout to make the plot interactive. Add another widget to select or deselect the smoother. Inspiration: https://demo.bokeh.org/weather